<a href="https://colab.research.google.com/github/cs20m016/cs6910-Assignment-3/blob/main/NoAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb -qq
import wandb

     |████████████████████████████████| 1.8MB 7.3MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 133kB 56.2MB/s 
     |████████████████████████████████| 174kB 58.2MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 


In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!unzip /content/drive/MyDrive/lexicons.zip > /dev/null

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
from keras.utils.vis_utils import plot_model

In [7]:
# Path to the data txt file on disk.
data_path = "/content/lexicons/hi.translit.sampled.train.tsv"
val_path = "/content/lexicons/hi.translit.sampled.dev.tsv"
test_path = "/content/lexicons/hi.translit.sampled.test.tsv"

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters.add(" ")
target_characters.add(" ")
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [9]:
val_input_texts = []
val_target_texts = []

with open(val_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)

In [10]:
test_input_texts = []
test_target_texts = []

with open(val_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: len(lines) - 1]:
    target_text, input_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    test_input_texts.append(input_text)
    test_target_texts.append(target_text)

In [11]:
print("Number of samples:", len(input_texts))
print("Number of validation samples:", len(val_input_texts))
print("Number of test samples:", len(test_input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 44204
Number of validation samples: 4358
Number of test samples: 4358
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [12]:
#Train Set
for i in range (5):
  index = random. randint(0,len(target_texts))
  print(input_texts[index],target_texts[index])

madonna 	मडोना

samvedak 	संवेदक

agrejon 	अग्रेजों

sambadhi 	संबधी

aansuon 	आंसुओं



In [13]:
#Validation Set
for i in range (5):
  index = random. randint(0,len(val_target_texts))
  print(val_input_texts[index],val_target_texts[index])

jagkar 	जागकर

niraj 	नीरज

kamzoriyon 	कमज़ोरियों

goyethe 	गोएथे

jamshedpur 	जमशेदपुर



In [14]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [15]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
print(np.shape(encoder_input_data),np.shape(decoder_input_data),np.shape(decoder_target_data))

(44204, 20, 27) (44204, 21, 66) (44204, 21, 66)


In [16]:
val_encoder_input_data = np.zeros((len(val_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
val_decoder_input_data = np.zeros((len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
val_decoder_target_data = np.zeros((len(val_input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
print(np.shape(val_encoder_input_data),np.shape(val_decoder_input_data),np.shape(val_decoder_target_data))

(4358, 20, 27) (4358, 21, 66) (4358, 21, 66)


In [17]:
test_encoder_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
print(np.shape(test_encoder_input_data))

(4358, 20, 27)


In [18]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [19]:
for i, (val_input_text, val_target_text) in enumerate(zip(val_input_texts, val_target_texts)):
    for t, char in enumerate(val_input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.0
    val_encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(val_target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        val_decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            val_decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    val_decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    val_decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [20]:
for i, (test_input_text, ) in enumerate(zip(test_input_texts, )):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token_index[char]] = 1.0
    test_encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

In [21]:
def RNN_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout):
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.SimpleRNN(latent_dim, return_state=True)
  encoder_outputs, state_h = encoder(encoder_inputs)

  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  decoder_gru = keras.layers.SimpleRNN(latent_dim, return_sequences=True,return_state=True)
  decoder_outputs,_ = decoder_gru(decoder_inputs, initial_state=state_h)
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer=optimizer_fn, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data))
  encoder_inputs = model.input[0]  # input_1
  encoder_outputs, state_h_enc = model.layers[2].output  # gru_1
  encoder_model = keras.Model(encoder_inputs, state_h_enc)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,))
  decoder_gru = model.layers[3]
  decoder_outputs, state_h_dec = decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)
  decoder_dense = model.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + [decoder_state_input_h], [decoder_outputs] + [state_h_dec])
  return model,encoder_model,decoder_model

In [22]:
def GRU_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout):
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.GRU(latent_dim, return_state=True)
  encoder_outputs, state_h = encoder(encoder_inputs)

  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True,return_state=True)
  decoder_outputs,_ = decoder_gru(decoder_inputs, initial_state=state_h)
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer=optimizer_fn, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data))
  # Define sampling models GRU
  # Restore the model and construct the encoder and decoder.
  encoder_inputs = model.input[0]  # input_1
  encoder_outputs, state_h_enc = model.layers[2].output  # gru_1
  encoder_model = keras.Model(encoder_inputs, state_h_enc)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,))
  decoder_gru = model.layers[3]
  decoder_outputs, state_h_dec = decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)
  decoder_dense = model.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + [decoder_state_input_h], [decoder_outputs] + [state_h_dec])
  return model,encoder_model,decoder_model

In [23]:
def LSTM_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout):
  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.LSTM(latent_dim, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs)
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  model.compile(optimizer=optimizer_fn, loss="categorical_crossentropy", metrics=["accuracy"])
  model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_data=([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data))
  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  encoder_inputs = model.input[0]  # input_1
  encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
  encoder_states = [state_h_enc, state_c_enc]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,))
  decoder_state_input_c = keras.Input(shape=(latent_dim,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  decoder_lstm = model.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
  return model,encoder_model,decoder_model

In [24]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [25]:
def decode_sequence(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + [states_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = h
    return decoded_sentence

In [26]:
def decode_sequence_lstm(input_seq,encoder_model,decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [27]:
def Run(epochs,latent_dim, batch_size, dropout, recc_dropout, cell_type, optimizer_fn):
  if cell_type=='LSTM':
    model,encoder_model,decoder_model = LSTM_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout)
  elif cell_type=='GRU':
    model,encoder_model,decoder_model = GRU_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout)
  elif cell_type=='RNN':
    model,encoder_model,decoder_model = RNN_model(epochs,latent_dim, batch_size, optimizer_fn, dropout, recc_dropout)
  valid = 0
  samples=1000
  for i in range(samples):
      seq_index=random.randint(0,len(test_input_texts))
      input_seq = test_encoder_input_data[seq_index : seq_index + 1]
      if cell_type=='LSTM':
        decoded_sentence = decode_sequence_lstm(input_seq,encoder_model,decoder_model)
      else:
        decoded_sentence = decode_sequence(input_seq,encoder_model,decoder_model)
      flag=1
      for (i,j) in zip(decoded_sentence,test_target_texts[seq_index]):
        if i!=j:
          flag=0
          break
      if flag==1:
        valid += 1
  return (valid/samples)*100

In [28]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'model_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epoch': {
            'values': [40]
        },
        'batch_size': {
            'values': [64]
        },
        'dropout':{
            'values': [0.1,0.01,0.0]
        },
        'recc_dropout':{
            'values': [0.1,0.01,0.0]
        },
        'latent_dim':{
            'values': [64,126,256]
        },
        'cell_type': {
            'values':['LSTM','RNN','GRU']
        },
        'optimizer_fn': {
            'values': ['adam','rmsprop']
        }
    }
}

In [29]:
sweep_id = wandb.sweep(sweep_config,entity="dl-assignment3",project="NoAttention")

Create sweep with ID: 0glucakm
Sweep URL: https://wandb.ai/dl-assignment3/NoAttention/sweeps/0glucakm


In [30]:
def train():
    config_defaults={
        'epoch':10,
        'batch_size':64,
        'dropout':0.01,
        'recc_dropout':0.1,
        'cell_type':'LSTM',
        'optimizer_fn':'rmsprop',
        }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    model_acc=Run(config.epoch,config.latent_dim, config.batch_size,config.dropout, config.recc_dropout, config.cell_type, config.optimizer_fn)  
    wandb.log({'model_accuracy': model_acc})
    return

In [ ]:
wandb.agent(sweep_id, train,count=3)

wandb: Agent Starting Run: 3ndmn1qb with config:
wandb: 	batch_size: 64
wandb: 	cell_type: RNN
wandb: 	dropout: 0.1
wandb: 	epoch: 40
wandb: 	latent_dim: 126
wandb: 	optimizer_fn: rmsprop
wandb: 	recc_dropout: 0.1
wandb: Currently logged in as: athirapt (use `wandb login --relogin` to force relogin)


Epoch 1/40
691/691 [==============================] - 23s 27ms/step - loss: 1.1825 - accuracy: 0.7209 - val_loss: 0.6515 - val_accuracy: 0.8266
Epoch 2/40
691/691 [==============================] - 18s 25ms/step - loss: 0.6362 - accuracy: 0.8266 - val_loss: 0.5724 - val_accuracy: 0.8412
Epoch 3/40
691/691 [==============================] - 18s 26ms/step - loss: 0.5409 - accuracy: 0.8469 - val_loss: 0.4909 - val_accuracy: 0.8608
Epoch 4/40
691/691 [==============================] - 18s 26ms/step - loss: 0.4849 - accuracy: 0.8594 - val_loss: 0.4606 - val_accuracy: 0.8670
Epoch 5/40
691/691 [==============================] - 18s 26ms/step - loss: 0.4492 - accuracy: 0.8673 - val_loss: 0.4517 - val_accuracy: 0.8665
Epoch 6/40
691/691 [==============================] - 18s 26ms/step - loss: 0.4221 - accuracy: 0.8739 - val_loss: 0.4546 - val_accuracy: 0.8653
Epoch 7/40
691/691 [==============================] - 18s 26ms/step - loss: 0.3987 - accuracy: 0.8797 - val_loss: 0.4171 - val_accuracy: